# Recommender ChatBot: LLM/Collaborative Filtering recommender ensemble

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

BASE_PATH    = '../..'
API_PATH     = f'{BASE_PATH}/chat-bot-api'
LIB_PATH     = f'{BASE_PATH}/lib'


os.environ['TMP_PATH']         = f'{BASE_PATH}/tmp'
os.environ['DATASET_PATH']     = f'{BASE_PATH}/datasets'
os.environ['WEIGHTS_PATH']     = f'{BASE_PATH}/weights'
os.environ['METRICS_PATH']     = f'{BASE_PATH}/metrics'
os.environ['MONGODB_URL']      = 'mongodb://0.0.0.0:27017'
os.environ['MONGODB_DATABASE'] = 'chatbot'
os.environ['CHROMA_HOST']      = '0.0.0.0'
os.environ['CHROMA_PORT']      = '9090'

In [3]:
import sys
sys.path.append(LIB_PATH)
sys.path.append(API_PATH)

import pytorch_common.util as pu
from app_context import AppContext

2024-02-25 16:57:48.559648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 16:57:49.030682: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-25 16:57:49.289283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-25 16:57:49.298897: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

<Figure size 640x480 with 0 Axes>

## Setup

In [4]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [5]:
ctx = AppContext()

2024-02-25 16:57:50,696 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2024-02-25 16:57:51,247 - INFO - Use pytorch device: cuda
2024-02-25 16:57:51,249 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-02-25 16:57:51,268 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


## Recommender

In [47]:
recommendations = await ctx.database_user_item_filtering_recommender.recommend(
    user_id            = "adrianmarino@gmail.com",
    text_query         = 'I want to see sci-fi movies',
    k_sim_users        = 10,
    max_items_by_user  = 10,
    text_query_limit   = 1500,
    min_rating_by_user = 4,
    not_seen           = True
)

2024-02-25 17:11:52,298 - INFO - Found 10 similar users
2024-02-25 17:11:52,303 - INFO - Found 451 similar users interactions
2024-02-25 17:11:52,303 - INFO - Select 103 similar users interactions (max by user: 10, min rating: 4)
2024-02-25 17:11:52,512 - INFO - Found 1500 items by text query
2024-02-25 17:11:52,521 - INFO - Select 20 similar user items by text query
2024-02-25 17:11:52,522 - INFO - Select 20 similar user unseen items
2024-02-25 17:11:52,526 - INFO - 39 movies seen by adrianmarino@gmail.com user


In [48]:
recommendations.show_seen()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
Poster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Genres,AdventureAnimationChildrenComedyFantasy,ActionAdventureSci-fi,ActionSci-fiThrillerImax,AdventureAnimationChildrenComedyFantasy,ActionAdventureAnimationChildrenComedy,DramaMysterySci-fiThriller,AdventureAnimationChildrenComedyDramaFantasy,ActionAdventureSci-fi,AdventureAnimationChildrenComedyFantasy,ActionAdventure,ActionCrimeDramaImax,AdventureAnimationChildrenComedyFantasyImax,ActionCrimeDramaMysterySci-fiThrillerImax,Sci-fiImax,ActionAdventureFantasyImax,DramaMysteryThriller,AdventureAnimationChildrenComedy,ActionAdventureSci-fiThrillerWar,MysteryThriller,ActionAdventureAnimationChildrenComedy,AnimationChildrenDrama,ActionSci-fi,ActionAdventureSci-fiImax,ActionAdventureSci-fi,AdventureAnimationChildrenComedyFantasy,AdventureAnimationComedy,DramaWar,ActionSci-fiThriller,ActionAdventureSci-fi,ActionAdventureSci-fi,ActionAdventureAnimationChildrenComedyFantasy,AdventureAnimationChildrenComedyFantasy,ActionAdventureSci-fiImax,ActionSci-fiImax,Comedy,ActionAdventureSci-fiThrillerImax,Comedy,ActionAdventureSci-fi,ComedyDramaRomance
Rating,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,2.0


In [49]:
recommendations.show(
    sort_by        = ['user_sim_weighted_pred_rating_score'],
    k              = 10
)

,0,1,2,3,4,5,6,7,8,9
Poster,,,,,,,,,,
Genres,ComedyDramaSci-fi,AdventureComedySci-fi,ActionComedySci-fi,DramaSci-fi,AdventureAnimationChildrenDrama,ActionSci-fiImax,ActionHorrorSci-fiThriller,ActionAdventureSci-fiThriller,DramaSci-fiThriller,AnimationChildrenComedyCrime
Mean Rating,4.117647,4.101064,3.786517,3.953947,3.995122,4.009524,4.045455,3.862832,3.976636,3.60241
Predicted Rating,4.75918,4.655716,4.494987,4.472497,4.469231,4.318829,4.299498,4.276339,4.261951,4.201691
User sim weighted rating score,0.989114,0.985401,0.909822,0.949791,0.959902,0.96314,0.972115,0.928116,0.955241,0.86623
User sim weighted predicted rating score,0.989114,0.967877,0.934463,0.929531,0.929066,0.897593,0.893892,0.888967,0.885773,0.87414
User Item Similarity,0.989114,0.989386,0.989386,0.989114,0.98934,0.989112,0.989462,0.98934,0.989114,0.990123


### Notes
* Movies seen by similar users weighted by user predicted rating.
* Ordered by **User sim weighted predicted rating score**.
* **User sim weighted predicted rating score** = similar_user_similarity(0..1) * user_predicted_rating (Normalize to 0..1)

In [50]:
recommendations.show(
    sort_by        = ['user_sim_weighted_rating_score'],
    k              = 10
)

,0,1,2,3,4,5,6,7,8,9
Poster,,,,,,,,,,
Genres,ComedyDramaSci-fi,AdventureComedySci-fi,ActionHorrorSci-fiThriller,ActionSci-fiImax,AdventureAnimationChildrenDrama,DramaSci-fiThriller,DramaSci-fi,MysterySci-fiThriller,MysterySci-fiThriller,ActionAdventureSci-fiThriller
Mean Rating,4.117647,4.101064,4.045455,4.009524,3.995122,3.976636,3.953947,3.941748,3.883268,3.862832
Predicted Rating,4.75918,4.655716,4.299498,4.318829,4.469231,4.261951,4.472497,4.181332,4.139088,4.276339
User sim weighted rating score,0.989114,0.985401,0.972115,0.96314,0.959902,0.955241,0.949791,0.947194,0.932925,0.928116
User sim weighted predicted rating score,0.989114,0.967877,0.893892,0.897593,0.929066,0.885773,0.929531,0.869324,0.860341,0.888967
User Item Similarity,0.989114,0.989386,0.989462,0.989112,0.98934,0.989114,0.989114,0.989462,0.989232,0.98934


### Notes
* Movies seen by similar users weighted by movie mean rating.
* Ordered by **User sim weighted rating score**.
* **User sim weighted rating score** = similar_user_similarity(0..1) * mean_movie_rating (Normalize to 0..1) 

In [51]:
recommendations.show(
    sort_by        = ['user_item_sim'],
    k              = 10
)

,0,1,2,3,4,5,6,7,8,9
Poster,,,,,,,,,,
Genres,AnimationChildrenComedyCrime,AnimationComedy,AdventureActionFantasy,MysterySci-fiThriller,ActionHorrorSci-fiThriller,ActionSci-fi,ActionComedySci-fi,AdventureComedySci-fi,ActionAdventureSci-fiThriller,AdventureAnimationChildrenDrama
Mean Rating,3.60241,2.85,3.181818,3.941748,4.045455,3.555556,3.786517,4.101064,3.862832,3.995122
Predicted Rating,4.201691,3.759929,4.19021,4.181332,4.299498,4.124599,4.494987,4.655716,4.276339,4.469231
User sim weighted rating score,0.86623,0.685307,0.765095,0.947194,0.972115,0.854327,0.909822,0.985401,0.928116,0.959902
User sim weighted predicted rating score,0.87414,0.782234,0.871752,0.869324,0.893892,0.857463,0.934463,0.967877,0.888967,0.929066
User Item Similarity,0.990123,0.990123,0.990123,0.989462,0.989462,0.989386,0.989386,0.989386,0.98934,0.98934


### Notes
* Movies seen by similar users.
* Ordered by **similar user similarity by movie**.
* take movies sample from each similar user, then assign similary and order by similary.